In [1]:
# MIT image
# Made by : HERO Kwon
# Date : 190108

import os
import numpy as np
import pandas as pd
import pickle
import gzip
import matplotlib.pyplot as plt
import math
import cmath

# data path_mi
path_csi = 'D:\\Data\\Wi-Fi_processed\\'
path_csi_np = 'D:\\Data\\Wi-Fi_processed_npy\\'
path_meta = 'D:\\Data\\Wi-Fi_meta\\'

In [2]:
# data info
df_info = pd.read_csv(path_meta+'data_subc_sig_v1.csv')
person_uid = np.unique(df_info['id_person'])
dict_id = dict(zip(person_uid,np.arange(len(person_uid))))
csi_time = 15000 #int(np.max(df_info['len']))
# parameters
max_value = np.max(df_info['max'].values)
#no_classes = len(np.unique(df_info['id_person']))
no_classes = len(dict_id)
csi_subc = 30
input_shape = (csi_time, csi_subc, 6)

# freq BW list
bw_list = pd.read_csv(path_meta+'wifi_f_bw_list.csv')

# avg Array
with open(path_meta + 'dict_avgcsi.pickle','rb') as f:
    dict_avg = pickle.load(f)

In [3]:
# make data generator
def gen_csi(df_info,dict_avg,id_num,len_num):
    for file in np.unique(df_info.id.values):
        # read sample data
        # load and uncompress.
        with gzip.open(path_csi+file+'.pickle.gz','rb') as f:
            data1 = pickle.load(f)
        data1_diff = data1 #np.diff(data1,axis=0)
        # zero pad
        pad_len = len_num - data1_diff.shape[0]
        data1_pad = np.pad(data1_diff,((0,pad_len),(0,0),(0,0),(0,0)),'constant',constant_values=0)

        # Label
        id_key = df_info[df_info.id==file]['id_person'].values[0].astype('int')
        data1_y = dict_id[id_key]
        
        arr_avg = dict_avg[id_key][0]
        data1_result = data1_pad - arr_avg
        
        yield(data1_result ,data1_y,id_key)

gen = gen_csi(df_info,dict_avg,no_classes,csi_time)
target_sig,target_lab,target_id = next(gen)

In [4]:
# 3D scan
m,n = 2,3
c =  299792458 # speed of light 
r = (160 + 160 + 164) * 0.01 # meter
#r = 1.64 #meter
d = 45 * 0.01 # meter
max_ch = 1#3
subc = 14

th_range,si_range = (10,10)
sig_mat = np.zeros([max_ch,csi_time,th_range,si_range])


In [5]:
!conda install accelerate

Solving environment: ...working... failed



UnsatisfiableError: The following specifications were found to be in conflict:
  - accelerate
  - anaconda==5.2.0=py36_3 -> numba==0.38.0=py36h830ac7b_0 -> llvmlite=0.23
  - anaconda==5.2.0=py36_3 -> numexpr==2.6.5=py36hcd2f87e_0
  - anaconda==5.2.0=py36_3 -> scikit-learn==0.19.1=py36h53aea1b_0
Use "conda info <package>" to see the dependencies for each package.

